In [2]:
pip install psycopg2-binary


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 1.7 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.6 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install sqlalchemy


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 3.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 2.8 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [14]:
#Define your connection parameters
hostname = 'localhost'
database = 'postgres'
username = 'postgres'
password = 'admin'
port_id = '5432'

#Step 1 : Extract data using psycopg2 and transform it with pandas 
try:
    conn = psycopg2.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port_id
    )

#print a message when the connection is established 
    print("\n\nConnection to the PostgreSQL database established successfully.\n\n")

#create a cursor object 
    cur = conn.cursor()

#Execute a query to fetch Data 
    cur.execute('SELECT * FROM customer_info')

#fetch the results
    records = cur.fetchall()

#extract column names from the cursor 
    colnames = [desc[0] for desc in cur.description]

#Transform the result into a dataframe
    df = pd.DataFrame(records, columns=colnames)
    print(df)

    cur.close()
except (Exception, psycopg2.DatabaseError) as error :
    print(error)
finally: 
    if conn is not None:
        conn.close()

#step 2 : Clean and Transform Data
df_cleaned = df.dropna(subset=['full_name'])
print("Cleaned DataFrame:")
print(df_cleaned)

def split_full_name(df_cleaned):
    first_names = []
    last_names = []

    for name in df_cleaned['full_name']:
        parts = name.split(' ',1)
        first_names.append(parts[0])
        last_names.append(parts[1] if len(parts) > 1 else '')

    df_cleaned['first_name'] = first_names
    df_cleaned['last_name'] = last_names

    df_cleaned = df_cleaned.drop(columns=['full_name'])
    return df_cleaned

df_cleaned = split_full_name(df_cleaned)
print("\n\nDataFrame after splitting Full Names:\n\n")
print(df_cleaned)


#Step 3 : Load the dataframe into PostgreSQL using SQLAlchemy

#Create SQLAlchemy engine 
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{hostname}:{port_id}/{database}')


#Load The Dataframe into the PostgreSQL Table
#The Table will be created if it does not exist or data will be appended if it does 

df_cleaned.to_sql('cleaned_customer_info', engine, index=False, if_exists='append')
print("Data loaded successfully into PostgreSQL.")



    



Connection to the PostgreSQL database established successfully.


    id          full_name   age
0    1  Kaustubh Salunkhe  26.0
1    2    Kishor Salunkhe  61.0
2    3    Manali Salunkhe  49.0
3    4    Sayali Salunkhe  29.0
4    5  Arnab Debadhikari  28.0
5    6      Rohit Divekar  25.0
6    7        Amey Chavan  31.0
7    8        Hariram Pal   NaN
8    9        Bipin Rawat  56.0
9   10        Kunal Patil   NaN
10  11     Kiran Salunkhe   NaN
11  12     Suraj Salunkhe   NaN
Cleaned DataFrame:
    id          full_name   age
0    1  Kaustubh Salunkhe  26.0
1    2    Kishor Salunkhe  61.0
2    3    Manali Salunkhe  49.0
3    4    Sayali Salunkhe  29.0
4    5  Arnab Debadhikari  28.0
5    6      Rohit Divekar  25.0
6    7        Amey Chavan  31.0
7    8        Hariram Pal   NaN
8    9        Bipin Rawat  56.0
9   10        Kunal Patil   NaN
10  11     Kiran Salunkhe   NaN
11  12     Suraj Salunkhe   NaN


DataFrame after splitting Full Names:


    id   age first_name    last_name
0 